In [40]:
# creating a tensor
import tensorflow as tf

tens=tf.constant([[2,3,4],[2,3,4]])

In [41]:
tens.shape,tens.dtype

(TensorShape([2, 3]), tf.int32)

In [42]:
tens[:,1:]

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[3, 4],
       [3, 4]])>

In [43]:
a=tens[:,1,tf.newaxis]
a

<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[3],
       [3]])>

In [44]:
tens @ tf.transpose(tens)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[29, 29],
       [29, 29]])>

In [45]:
tf.transpose(tens)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[2, 2],
       [3, 3],
       [4, 4]])>

In [46]:
tens

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[2, 3, 4],
       [2, 3, 4]])>

In [47]:
tens+10

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[12, 13, 14],
       [12, 13, 14]])>

In [48]:
tens @ tf.transpose(tens)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[29, 29],
       [29, 29]])>

In [49]:
import numpy as np
a=np.array([[23,24,25],[26,27,28]])
a.T

array([[23, 26],
       [24, 27],
       [25, 28]])

In [50]:
a=np.array([2,4,5])
b=tf.constant(a)
b

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([2, 4, 5])>

In [51]:
type(b)

tensorflow.python.framework.ops.EagerTensor

In [52]:
tf.constant(2) + tf.constant(54)

<tf.Tensor: shape=(), dtype=int32, numpy=56>

In [53]:
a=tf.constant(40,dtype=tf.float64)
b=tf.constant(2.)
c=b+tf.cast(a,tf.float32)

In [54]:
c

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

In [55]:
v=tf.Variable([[1,2,3],[4,5,6]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

In [56]:
v.assign(2*v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[ 2,  4,  6],
       [ 8, 10, 12]])>

In [57]:
v[1,1].assign(32)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[ 2,  4,  6],
       [ 8, 32, 12]])>

In [58]:
v[:,1].assign(90)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[ 2, 90,  6],
       [ 8, 90, 12]])>

In [59]:
v.scatter_nd_update(indices=[[0,0],[1,2]],updates=[100,200])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[100,  90,   6],
       [  8,  90, 200]])>

In [60]:
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[100,  90,   6],
       [  8,  90, 200]])>

In [61]:
tf.__version__

'2.11.0'

In [62]:
# lets define a huberman loss
def huber_fn(y_true,y_pred):
    error=y_true-y_pred
    is_small_error=tf.abs(error)<1
    square_loss=tf.square(error)/2
    linear_loss=tf.abs(error)- 0.5
    return tf.where(is_small_error,square_loss,liear_loss)

In [63]:
# hubberman loss with threshold loss
def create_hubber(threshold=1.0):
    def hubber_fn(y_true,y_pred):
        error=tf.abs(y_true,y_pred)
        is_small_error=tf.abs(error)<threshold
        square_loss=tf.square(error)/2
        linear_loss=threshold*tf.abs(error)-threshold**2/2
        return tf.where(is_small_error,sqaure_error,linear_loss)
    

In [64]:
#lets make it so that can adjust it when u save the model
import keras
class HubberLoss(keras.losses.Loss):
    def __init__(self,threshold=1.0,**kwargs):
        self.threshold=threshold
        super().__init__(**kwargs)
    def call(self,y_true,y_pred):
        error=tf.abs(y_true,y_pred)
        is_small_error=tf.abs(error)<self.threshold
        square_loss=tf.square(error)/2
        linear_loss=self.threshold*tf.abs(error)-self.threshold**2/2
        return tf.where(is_small_error,sqaure_error,linear_loss)
    def get_config(self):
        base_config=super().get_config()
        return {**base_config,"threshol":self.threshold}

In [65]:
precision=keras.metrics.Precision()

In [66]:
precision([0, 1, 1, 1, 0, 1, 0, 1],[1, 1, 0, 1, 0, 1, 0, 1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [67]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([1.], dtype=float32)>]

In [68]:
precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [69]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

# custome metric...its the same as loss

In [70]:
class HubberMetric(keras.metrics.Metric):
    def __init(self,threshold=1.0,*kwargs):
        super().__init__(*kwargs)
        self.threshold=threshold
        self.hubber_fn=create_hubber(threshold)
        self.total=self.add_weight("total",initializer="zeros")
        self.count=self.add_weight("count",initializer="zeros")
    def update_state(self,y_true,y_pred,sample_weight=None):
        metric=self.hubber(y_truey_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true),tf.float32))
    def result(self):
        return self.total/self.count
    def get_config(self):
        base_config=super().get_config()
        return {**base_config,"threshold":self.threshold}

In [71]:
#custom layers
exponentia_layer=keras.layers.Lambda(lambda x:tf.exp(x))

In [72]:
class mydense(keras.layers.Layer):
    def __init__(self,units,activation=None,**kwargs):
        super().__init__(**kwargs)
        self.units=units,
        self.activation=keras.activations.get(activation)  
    def build(self,batch_input_shape):
        self.kernel=self.add_weight(
            name="kernel",shape=[batch_input_shape[-1],self.units],
            initializer="glorot_normal"
        )
        self.bias=self.add_weight(
            name="bias",shape=[self.units],initializer="zeros"
        )
        super().build(batch_input_shape)
    def call(self,x):
        return self.activation(x @ self.kernel + self.bias)
    def compute_output_shape(self,batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[-1] + [self.units])
    def get_config(self):
        base_config=super().get_config()
        return {**base_config,"units":self.units,"activation":keras.activations.serialize(self.activation)}

# making custom model

In [73]:
# lets make a residual block of code:
class residualblock(keras.layers.Layer):
    def __init__(self,n_layers,n_neurons,**kwargs):
        super().__init__(**kwargs)
        self.hidden=[
                     keras.layers.Dense(n_neurons,activation="elu",kernel_initializer="he_normal") 
                     for _ in range(n_layers)
                    ]
    
    def call(self,inputs):
        z=inputs
        for layer in self.hidden:
            z=layer(z)
        return inputs+z

In [74]:
class residualregressor(keras.Model):
    #make the architecture of the model at the constructor
    def __init__(self,output_dim,**kwargs):
        super().__init__(**kwargs)
        self.hidden1=keras.layers.Dense(30,activation="elu",kernel_initializer="he_normal")  #this is the first layer
        self.block1=residualblock(2,30)#this is the residual block
        self.block2=residualblock(2,30) #this is the residual block
        self.out=keras.layers.Dense(output_dim)  #this is the output layer
        
    def call(self,inputs):
        z=self.hidden1(inputs)
        for _ in range(4):
            z=self.block1(inputs)
        z=self.block2(z)
        return self.out(z)

In [75]:
t1=residualregressor(4)

# making a custom loss functon based on the weight not the predicted outputs

In [76]:
class reconstructregressor(keras.Model):
    def __init__(self,output_dim,**kwargs):
        super().__init__(**kwargs)
        self.hidden=[keras.layers.Dense(30,activation="relu",kernel_initializer="lecun_normal")
                     for _ in range(5)
                    ]
        self.out=keras.layers.Dense(output_dim)
    
    def build(self,batch_input_shape):
        n_inputs=batch_input_shape[-1]
        self.reconstruct=keras.layers.Dense(n_inputs)
        super.build(batch_input_shape)
    
    def call(self,inputs):
        z=inputs
        for layer in self.hidden:
            z=layer(z)
            
        reconstruction=self.reconstruct(z)
        recon_loss=tf.reduce_mean(tf.square(reconstruction-inputs))
        self.add_loss(0.05*recon_loss)
        return self.out(z)
    

In [77]:
def f(w1,w2):
    return 3*w1**2+2*w1*w2

In [79]:
w1,w2=tf.Variable(5.),tf.Variable(3.)
with tf.GradientTape() as tape:
    z=f(w1,w2)
gradient=tape.gradient(z,[w1,w2])

gradient

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [85]:
c1, c2 = tf.constant(5.), tf.constant(3.)
with tf.GradientTape() as tape:
    tape.watch(c1)  #it make the gradient watch the constant vairables
    tape.watch(c2)
    
    z = f(c1, c2)
gradients = tape.gradient(z, [c1, c2]) 
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]